In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil import tz
import pytz
from scipy import stats
%matplotlib inline
pd.set_option("display.max_rows", 15)

In [2]:
energy = pd.read_csv("Energy.csv")
daytimes = pd.read_csv("Daytimes.csv")
weather = pd.read_csv("Weather.csv")
weather

,air_temp_set_1,date_time,pressure_set_1d,relative_humidity_set_1,weather_condition_set_1d,wind_speed_set_1
0,14.4,Y2013M10D14H00M51,99775.27,77.94,Partly Cloudy,4.63
1,12.0,Y2013M10D14H01M43,99841.78,87.57,Mostly Cloudy,1.54
2,12.2,Y2013M10D14H01M51,99841.78,86.42,Mostly Cloudy,3.09
3,12.2,Y2013M10D14H02M51,99875.05,89.96,Overcast,0.00
4,12.8,Y2013M10D14H03M51,99908.31,86.49,Overcast,0.00
5,12.8,Y2013M10D14H04M51,99941.56,86.49,Fog,1.54
6,12.0,Y2013M10D14H05M49,99941.56,87.57,Fog,1.54
...,...,...,...,...,...,...
129388,18.0,Y2017M05D09H19M35,100367.33,67.99,NaN,3.09
129389,18.0,Y2017M05D09H19M40,100367.33,67.99,NaN,3.60


In [3]:
#convert UTC time to local time so that sunsise and sunset times are on the same date.
local = pytz.timezone("US/Pacific")

weather.date_time = pd.to_datetime(weather.date_time, format='Y%YM%mD%dH%HM%M',utc = True)
weather.date_time = weather.date_time.dt.tz_localize(pytz.utc).dt.tz_convert(local)

daytimes.sunrise = pd.to_datetime(daytimes.sunrise, format='%Y-%m-%dT%H:%M:%S')
daytimes.sunset = pd.to_datetime(daytimes.sunset, format='%Y-%m-%dT%H:%M:%S')
daytimes.sunset = daytimes.sunset.dt.tz_localize(pytz.utc).dt.tz_convert(local)
daytimes.sunrise = daytimes.sunrise.dt.tz_localize(pytz.utc).dt.tz_convert(local)

#weather.head(), daytimes.head()

In [4]:
#remove nightime weather readings by removing readings taken before sunrise and after sunset.
daytimes["Date"] = daytimes.sunrise.dt.date
weather["Date"]= weather.date_time.dt.date

merged = weather.merge(daytimes, on="Date")
daytime_weather = merged.loc[(merged.sunrise.dt.time <= merged.date_time.dt.time)& (merged.sunset.dt.time >= merged.date_time.dt.time), :]

In [5]:
#Aggregate the quantitative variables accross days
weather_quantitative = daytime_weather.groupby(daytime_weather.date_time.dt.date).mean()

In [6]:
weather_quantitative.head()

,air_temp_set_1,pressure_set_1d,relative_humidity_set_1,wind_speed_set_1,day_length
date_time,,,,,
2013-10-14,21.563636,99778.295455,40.475455,3.740909,40786
2013-10-15,24.841667,99542.463333,27.645833,3.215000,40660
2013-10-16,24.836364,99914.350909,23.462727,3.085455,40533
2013-10-17,21.409091,99766.200000,32.283636,4.020909,40407
2013-10-18,22.281818,99760.157273,37.134545,2.431818,40283


In [7]:
#Aggregate categorical weather condition at the day level:
#the most frequent weather condition of the day is set as that day's weather condition
mostfreq_weather = {
    "Date":[],
    "Weather_overall": []
}
days = daytime_weather.groupby(daytime_weather.date_time.dt.date)
for day in days:
    mostfreq_weather["Date"].append(day[0])
    weather_label = list(day[1]["weather_condition_set_1d"].mode())
    if len(weather_label)>0:
        mostfreq_weather["Weather_overall"].append(weather_label[0])
    else:
        mostfreq_weather["Weather_overall"].append(np.nan)
agg_weather_condition = pd.DataFrame(mostfreq_weather)


In [8]:
#Create quantitative daily values to express the cloud cover.
weather_cats = daytime_weather.groupby(daytime_weather.date_time.dt.date)["weather_condition_set_1d"].value_counts(dropna = False).unstack()
weather_cats.head()

weather_condition_set_1d,nan,Blowing Dust,Clear,Fog,Haze,"Haze,Blowing Dust",Heavy Rain,"Heavy Rain,Fog","Heavy Rain,Hail","Heavy Rain/Thunderstorm,Fog",...,Partly Cloudy,Rain,"Rain,Fog",Sky Obscured,Smoke,"Smoke,Blowing Dust",Thunder,Thunderstorm,"Thunderstorm,Fog","Thunderstorm,Hail"
date_time,,,,,,,,,,,,,,,,,,,,,
2013-10-14,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-15,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-16,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-17,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-18,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
weather_cats["total"] = weather_cats.sum(axis = 1)
weather_cats.columns

Index([                          nan,                'Blowing Dust',
                             'Clear',                         'Fog',
                              'Haze',           'Haze,Blowing Dust',
                        'Heavy Rain',              'Heavy Rain,Fog',
                   'Heavy Rain,Hail', 'Heavy Rain/Thunderstorm,Fog',
                        'Light Rain',              'Light Rain,Fog',
           'Light Rain/Thunderstorm',                'Mostly Clear',
                     'Mostly Cloudy',                    'Overcast',
                     'Partly Cloudy',                        'Rain',
                          'Rain,Fog',                'Sky Obscured',
                             'Smoke',          'Smoke,Blowing Dust',
                           'Thunder',                'Thunderstorm',
                  'Thunderstorm,Fog',           'Thunderstorm,Hail',
                             'total'],
      dtype='object', name='weather_condition_set_1d')

In [10]:
weather_type_percentages = pd.DataFrame()
for column in weather_cats.loc[:, "Blowing Dust":"Thunderstorm,Hail"].columns:
    name = "pct_"+column
    weather_type_percentages[name] =( weather_cats[column] / weather_cats.total)*100

In [11]:
weather_type_percentages.head()

,pct_Blowing Dust,pct_Clear,pct_Fog,pct_Haze,"pct_Haze,Blowing Dust",pct_Heavy Rain,"pct_Heavy Rain,Fog","pct_Heavy Rain,Hail","pct_Heavy Rain/Thunderstorm,Fog",pct_Light Rain,...,pct_Partly Cloudy,pct_Rain,"pct_Rain,Fog",pct_Sky Obscured,pct_Smoke,"pct_Smoke,Blowing Dust",pct_Thunder,pct_Thunderstorm,"pct_Thunderstorm,Fog","pct_Thunderstorm,Hail"
date_time,,,,,,,,,,,,,,,,,,,,,
2013-10-14,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-15,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-16,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-17,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-18,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
weather_quantitative = weather_quantitative.merge(agg_weather_condition, right_on="Date", left_index=True)

In [13]:

final_weather_data = weather_quantitative.merge(weather_type_percentages, right_index=True, left_on="Date")
final_weather_data

,air_temp_set_1,pressure_set_1d,relative_humidity_set_1,wind_speed_set_1,day_length,Date,Weather_overall,pct_Blowing Dust,pct_Clear,pct_Fog,...,pct_Partly Cloudy,pct_Rain,"pct_Rain,Fog",pct_Sky Obscured,pct_Smoke,"pct_Smoke,Blowing Dust",pct_Thunder,pct_Thunderstorm,"pct_Thunderstorm,Fog","pct_Thunderstorm,Hail"
0,21.563636,99778.295455,40.475455,3.740909,40786,2013-10-14,Clear,NaN,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24.841667,99542.463333,27.645833,3.215000,40660,2013-10-15,Clear,NaN,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24.836364,99914.350909,23.462727,3.085455,40533,2013-10-16,Clear,NaN,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21.409091,99766.200000,32.283636,4.020909,40407,2013-10-17,Clear,NaN,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22.281818,99760.157273,37.134545,2.431818,40283,2013-10-18,Clear,NaN,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,14.714286,99865.542143,70.527857,1.947857,40158,2013-10-19,Clear,NaN,57.142857,35.714286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,14.100000,99703.212778,76.987222,1.771667,40034,2013-10-20,Haze,NaN,22.222222,33.333333,...,5.555556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,20.500000,100533.527378,65.505488,2.841890,49302,2017-05-03,Haze,NaN,3.954802,11.864407,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1298,16.400578,100585.809769,80.101156,2.988960,49410,2017-05-04,Clear,NaN,42.857143,40.109890,...,1.098901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
energy["Dateobs"] = pd.to_datetime(energy[['Year', 'Month', 'Day']])

energy.Date = energy.Dateobs.dt.date
final_data = energy.merge(final_weather_data, right_on="Date", left_on="Date")


In [15]:
#The first day was an outlier since we finished installing the solar panels that afternoon so it is dropped. 
final_data.drop(0, axis = 0, inplace=True)


In [16]:
final_data.columns

Index(['Date/Time', 'Energy Produced (Wh)', 'Year', 'Month', 'Day', 'Date',
       'Dateobs', 'air_temp_set_1', 'pressure_set_1d',
       'relative_humidity_set_1', 'wind_speed_set_1', 'day_length',
       'Weather_overall', 'pct_Blowing Dust', 'pct_Clear', 'pct_Fog',
       'pct_Haze', 'pct_Haze,Blowing Dust', 'pct_Heavy Rain',
       'pct_Heavy Rain,Fog', 'pct_Heavy Rain,Hail',
       'pct_Heavy Rain/Thunderstorm,Fog', 'pct_Light Rain',
       'pct_Light Rain,Fog', 'pct_Light Rain/Thunderstorm', 'pct_Mostly Clear',
       'pct_Mostly Cloudy', 'pct_Overcast', 'pct_Partly Cloudy', 'pct_Rain',
       'pct_Rain,Fog', 'pct_Sky Obscured', 'pct_Smoke',
       'pct_Smoke,Blowing Dust', 'pct_Thunder', 'pct_Thunderstorm',
       'pct_Thunderstorm,Fog', 'pct_Thunderstorm,Hail'],
      dtype='object')

In [17]:
!pwd

/home/tmgerrit/Data301/Final Project


In [18]:
final_data.to_csv("/home/tmgerrit/Data301/Final Project/cleaned_data.csv", index=False)